In [3]:
import sqlite3
import pandas as pd
import plotly.express as px

cnx = sqlite3.connect(r'C:\Users\Yoric\Uni\dataviz-2023\dataset\nba.sqlite')

query = '''
SELECT * 
FROM Game
'''

query2 = '''
    SELECT 
        SEASON_ID AS season,
        COUNT(DISTINCT GAME_ID) AS total_game_count,
        SUM(CASE WL_HOME 
                WHEN'W' THEN 1
                ELSE 0
            END) AS home_win_count
    FROM Game
    GROUP BY SEASON_ID
'''

df_summary = pd.read_sql_query(query2, cnx)
df_summary['home_win_percentage'] = round(100*df_summary['home_win_count'] / df_summary['total_game_count'], 2)
df_summary['season'] = df_summary['season'].astype(str).str[1:]


px.line(df_summary, y="home_win_percentage", x="season", title="Overall win percentage from 1946-2022")